scrape all the possible
https://royalsocietypublishing.org/doi/10.1098/rstl.1809.0001
links (number combinations after rstl)
(change the numbers, program reasonable ones to cycle through)
(1600 to 1899 or 1600 to 1886)
(0000 to 0100)

add epdf to the found links to get the pdf itself, scrape it:
title, author, publisher, year, content {as a long string}
{can also maybe skip the 1st step and do the cycling to epdf}

clean the text so it is non-punctuated (or search through it without caring ab punctuation)

do post-processing on each scraped column to see if the content has words of interest, create a new dataframe with each row containing each instance + metadata + 30 word context

remove exact duplicates

In [2]:
#ToDO 
# - make it so the crawler works and isnt immediately caught
# - make it so that it can see all available URLS through only checking the HEAD before moving on to making a request (if brute forcing)
# - make it so that the crawler downloads the pdf locally, as well as the html of the pre-website (to get the metadata)
# - give the crawler rules and test how a CrawlerSpider works
# link = response.xpath('//a[contains(@class, "next-posts-link")]/a/@href').get() --extract the link forward?

import scrapy
from scrapy.spiders import CrawlSpider, Rule
from scrapy.linkextractors import LinkExtractor

class RoyalSocietySpider(CrawlSpider):
    name = 'royalsociety_spider'
    allowed_domains = ['royalsocietypublishing.org']
    start_urls = ['https://royalsocietypublishing.org/loi/rstl/group/c1600.d1690.y1692']

    custom_settings = {
    # Where the item data should be saved
    'FEEDS': {'royalsociety.csv': {'format': 'csv', 'overwrite': True}},
    
    # Headers: This might have to be site specific to get past the 403 scraping measures
    'HEADERS': {
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:101.0) Gecko/20100101 Firefox/101.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'DNT': '1',
        'Upgrade-Insecure-Requests': '1',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'none',
        'Sec-Fetch-User': '?1',
        'Connection': 'keep-alive',
        'If-None-Match': '"f0267-ybK8wNq/yADu0m5N1CYhPqrXfaY"',
        }
    }

    rules = (
        #stay and follow links on rstl, dont go to journals or for-readers
        Rule(LinkExtractor(allow=(r"rstl",), deny=(r"journals", r"/rstl/for-readers"))),
        # when a link is the pre-part of an article, extract it, do a specific parse from which the pdf will also be gained
        Rule(LinkExtractor(allow=(r"g/doi/10.1098/rstl",)), callback="parse_intro_page"),
    )

    def parse_intro_page(self, response):
        self.logger.info("Hi, this is an article page! %s", response.url)

        item = scrapy.Item() #creating an item

        #scraping all the info from the article page and placing it in an item
        item['author'] = response.css('a.author-name::attr(title)').get()
        item['author'] = item['author'] if item['author'] else 'Unknown Author' #if author doesnt exist, its unknown author

        item['publisher'] = response.css('meta[name="dc.Publisher"]::attr(content)').get()
        item['date'] = response.css('meta[name="dc.Date"]::attr(content)').get()
        item['identifier'] = response.css('meta[name="dc.Identifier"]::attr(content)').get()
        item['og_url'] = response.css('meta[property="og:url"]::attr(content)').get()
        item['title'] = response.css('meta[name="dc.Title"]::attr(content)').get()
        item['language'] = response.css('meta[name="dc.Language"]::attr(content)').get()
        item['pdf_link'] = response.css('a:contains("View PDF")::attr(href)').get() #gets the pdf link

        #save the start pages with their url and title as html, just for now
        if item['og_url']: 
            og_url_without_quotes = item['og_url'].replace("'", "")
            title = item['title']
            filename = f"{title}_{og_url_without_quotes.split('/')[-1]}.html"

            with open(filename, 'wb') as f:
                f.write(response.body)
                self.log(f'Saved file {filename}')
        else:
            self.log('No og:url found in the item; not saved')

        yield item

        if item['pdf_link'] is not None: #if there is a link to the pdf, follow it
            return response.follow(
                item['pdf_link'], self.parse_pdf, cb_kwargs=dict(item=item)
            )

    def parse_pdf(self, response, item):
        #download pdf https://royalsocietypublishing.org/doi/pdf/10.1098/rstl.1809.0001?download=true add ?download=true to the link
        pdf_url = item['pdf_link']
        


In [3]:
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings

# Initialize CrawlerProcess
process = CrawlerProcess()

# Start the spider
process.crawl(RoyalSocietySpider)
process.start()

2024-04-09 10:02:43 [scrapy.utils.log] INFO: Scrapy 2.11.1 started (bot: scrapybot)
2024-04-09 10:02:43 [scrapy.utils.log] INFO: Versions: lxml 4.9.2.0, libxml2 2.9.12, cssselect 1.2.0, parsel 1.9.1, w3lib 2.1.2, Twisted 24.3.0, Python 3.9.10 (tags/v3.9.10:f2f3f53, Jan 17 2022, 15:14:21) [MSC v.1929 64 bit (AMD64)], pyOpenSSL 24.1.0 (OpenSSL 3.2.1 30 Jan 2024), cryptography 42.0.5, Platform Windows-10-10.0.22631-SP0
2024-04-09 10:02:43 [scrapy.addons] INFO: Enabled addons:
[]
2024-04-09 10:02:43 [py.warnings] WARNING: c:\Users\Igiba\.pyenv\pyenv-win\versions\3.9.10\lib\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See t

In [ ]:
#testing the logic (this is brute forcing though)
year_range = [1600, 1899]
issue_range = [0, 100]
for year in range(year_range[0], year_range[1] + 1):
            for issue in range(issue_range[0], issue_range[1] + 1):
                issue_formatted = str(issue).zfill(4)
                url = f'https://royalsocietypublishing.org/doi/epdf/10.1098/rstl.{year}.{issue_formatted}'
                print(url)  # Print the generated URL